In [8]:
import pandas as pd 
import numpy as np 
from scipy import stats
from sklearn import metrics
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.decomposition import PCA
import xgboost as xgb
from xgboost import XGBClassifier 
from xgboost import XGBRegressor
from xgboost import plot_importance
import matplotlib.pylab as plt
from matplotlib import pyplot
import mlxtend
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline
from sklearn.linear_model import LogisticRegression  
import os

In [10]:
# 1. Find the start of the ball data 
df = pd.DataFrame()
venueMap = dict()

for filename in os.listdir("CricSheet/odi_csv_male/"):    
    filename = 'CricSheet/odi_csv_male/'+filename

#     file = open("CricSheet/odi_csv_male/" + filename)
    file = open(filename)
    
    print(filename)
    rows_to_skip = 0
    nrows = 0
    total_rows=0
    venueSeen = False 

    for line in file:
        if "venue" in line:
            if venueSeen == False:
                infoArray = line.split(',')
                venue = infoArray[2].strip('"')
                venue = venue.strip()
                venueSeen = True
#             print(venue)
        if not line.startswith('ball'):
            rows_to_skip += 1
        else:
            break
    file.seek(0)

    for line in file:
        if not line.startswith('2',5,6):
            nrows+=1
        total_rows+=1
    
    file.seek(0)
    nrows=nrows-rows_to_skip
    

    # 2. Create the DataFrame
    df1 = pd.read_csv(
        filepath_or_buffer = file,
        skiprows = rows_to_skip,
        names = ["Ball","Innings", "Balls", "Bat Team", "Batsman","Non_Striker","Bowler","BatsmanScore", "Extras", "Catch", "Catcher"],
        usecols = range(1,11),
        nrows=nrows
    )
   
    

    # 3. Modify columns
    over_col = []
    ball_col = []
    for num in df1['Balls']:
        num1, num2 = str(num).split('.')
        over_col.append(num1)
        ball_col.append(num2)
    df1['Over*'] = list(map(int,over_col))
    df1['Ball*'] = list(map(int,ball_col))
    df1.drop('Balls', axis=1, inplace=True)

    ball=1
    balls=[]
    for i in df1['Ball*']:
        balls.append(ball)
        ball+=1
    df1['Total balls*']=balls

    df1['Runs*'] = df1['BatsmanScore'].astype('int32') + df1['Extras'].astype('int32')
    Runs= list(df1['Runs*'])

    runs=0
    run_sum=[]
    for run in df1['Runs*']:
        runs+=int(run)
        run_sum.append(runs)
    df1['Total Runs*'] = run_sum   

    rr=0
    run_rate =[]
    for i in range(len(Runs)):
        over=df1['Over*'].iloc[i]
        ball2=df1['Ball*'].iloc[i]
        tot_runs= df1['Total Runs*'].iloc[i]
        rr= over*6 + ball2
        rr = tot_runs/rr
        rr*=6
        run_rate.append(round(rr,2))
    df1['Run rate*'] = run_rate

    wicket=0
    wickets=[]
    one_wicket=[]
    for out in df1['Catch']:
        if(not pd.isnull(out)):
            wicket+=1
            one_wicket.append(1)
        else:
            one_wicket.append(0)
        wickets.append(wicket)

    df1['Wickets*'] = wickets
    df1['Game ID'] = filename[-11:-4]
    df1['Venue'] = venue
    endOfDF1 = df1.tail(1)
#     print(endOfDF)
    
    if venue in venueMap.keys():
        print(venue + "in keys")
        venueMap[venue].append(endOfDF1['Total Runs*'].iloc[0])
    else:
        print(venue + "not in keys")
        venueMap[venue] = [endOfDF1['Total Runs*'].iloc[0]]
    

#     run_sum5=0
#     runs5=[]
#     Wickets= one_wicket
#     wicket_sum5=0
#     wickets5=[]
#     rr5=[]
#     for i in range(len(Runs)):
#         if(i<30):
#             wickets5.append(Wickets[i])
#             runs5.append(run_sum[i])
#             rr5.append(df1['Run rate*'].iloc[i])
#         else:
#             wicket_sum5= np.sum(Wickets[i-29:i+1])
#             wickets5.append(wicket_sum5)
#             run_sum5= np.sum(Runs[i-29:i+1])
#             runs5.append(run_sum5)
#             temp= runs5[i]/5
#             rr5.append(round(temp,2))
#     df1['Runs_last_5*']=runs5
#     df1['Wickets_last_5*']=wickets5
#     df1['RR_last_5*']=rr5

    rows_to_skip+=nrows

    # 1. Find the start of the ball data
    # filename = 'CricSheet/odi_csv_male/225171.csv'
    file = open(filename)

    # 2. Create the DataFrame
    df2 = pd.read_csv(
        filepath_or_buffer = file,
        skiprows = rows_to_skip,
        names = ["Ball","Innings", "Balls", "Bat Team", "Batsman","Non_Striker","Bowler","BatsmanScore", "Extras", "Catch", "Catcher"],
        usecols = range(1,11),
        nrows=nrows
    )

    if(df2.empty):
        continue

    # 3. Modify columns
    over_col = []
    ball_col = []
    for num in df2['Balls']:
        num1, num2 = str(num).split('.')
        over_col.append(num1)
        ball_col.append(num2)
    df2['Over*'] = list(map(int,over_col))
    df2['Ball*'] = list(map(int,ball_col))
    df2.drop('Balls', axis=1, inplace=True)

    ball=1
    balls=[]
    for i in df2['Ball*']:
        balls.append(ball)
        ball+=1
    df2['Total balls*']=balls

    df2['Runs*'] = df2['BatsmanScore'].astype('int32') + df2['Extras'].astype('int32')
    Runs= list(df2['Runs*'])

    runs=0
    run_sum=[]
    for run in df2['Runs*']:
        runs+=int(run)
        run_sum.append(runs)
    df2['Total Runs*'] = run_sum   

    rr=0
    run_rate =[]
    for i in range(len(Runs)):
        over=df2['Over*'].iloc[i]
        ball2=df2['Ball*'].iloc[i]
        tot_runs= df2['Total Runs*'].iloc[i]
        rr= over*6 + ball2
        rr = tot_runs/rr
        rr*=6
        run_rate.append(round(rr,2))
    df2['Run rate*'] = run_rate

    wicket=0
    wickets=[]
    one_wicket=[]
    for out in df2['Catch']:
        if(not pd.isnull(out)):
            wicket+=1
            one_wicket.append(1)
        else:
            one_wicket.append(0)
        wickets.append(wicket)

    df2['Wickets*'] = wickets
    df2['Game ID'] = filename[-11:-4]
    df2['Venue'] = venue
    endOfDF2 = df2.tail(1)
    
    if venue in venueMap.keys():
        print(venue)
        venueMap[venue].append(endOfDF2['Total Runs*'].iloc[0])
    else:
        print(venue)
        venueMap[venue] = [endOfDF2['Total Runs*'].iloc[0]]


#     run_sum5=0
#     runs5=[]
#     Wickets= one_wicket
#     wicket_sum5=0
#     wickets5=[]
#     rr5=[]
#     for i in range(len(Runs)):
#         if(i<30):
#             wickets5.append(Wickets[i])
#             runs5.append(run_sum[i])
#             rr5.append(df2['Run rate*'].iloc[i])
#         else:
#             wicket_sum5= np.sum(Wickets[i-29:i+1])
#             wickets5.append(wicket_sum5)
#             run_sum5= np.sum(Runs[i-29:i+1])
#             runs5.append(run_sum5)
#             temp= runs5[i]/5
#             rr5.append(round(temp,2))
#     df2['Runs_last_5*']=runs5
#     df2['Wickets_last_5*']=wickets5
#     df2['RR_last_5*']=rr5

    
    if ("Afghanistan" or "Pakistan" or "India" or "West Indies" or "Austrailia" or "England" or "Bangladesh" or "New Zealand" or "South Africa" or "Sri Lanka") in (endOfDF1['Bat Team'].iloc[0] or endOfDF2['Bat Team'].iloc[0]): 

        if df.empty:
            frames = [df1,df2]

        else:
            frames = [df1,df2,df]
        df = pd.concat(frames)

CricSheet/odi_csv_male/1000887.csv
Brisbane Cricket Groundnot in keys
Brisbane Cricket Ground
CricSheet/odi_csv_male/1000889.csv
Melbourne Cricket Groundnot in keys
Melbourne Cricket Ground
CricSheet/odi_csv_male/1000891.csv
Western Australia Cricket Association Groundnot in keys
Western Australia Cricket Association Ground
CricSheet/odi_csv_male/1000893.csv
Sydney Cricket Groundnot in keys
Sydney Cricket Ground
CricSheet/odi_csv_male/1000895.csv
Adelaide Ovalnot in keys
Adelaide Oval
CricSheet/odi_csv_male/1001371.csv
Sydney Cricket Groundin keys
Sydney Cricket Ground
CricSheet/odi_csv_male/1001373.csv
Manuka Ovalnot in keys
Manuka Oval
CricSheet/odi_csv_male/1001375.csv
Melbourne Cricket Groundin keys
Melbourne Cricket Ground
CricSheet/odi_csv_male/1003271.csv
Grange Cricket Club Groundnot in keys
CricSheet/odi_csv_male/1003273.csv
Grange Cricket Club Groundin keys
Grange Cricket Club Ground
CricSheet/odi_csv_male/1004283.csv
Grange Cricket Club Groundin keys
Grange Cricket Club Grou

Harare Sports Clubin keys
Harare Sports Club
CricSheet/odi_csv_male/1080630.csv
ICC Global Cricket Academynot in keys
ICC Global Cricket Academy
CricSheet/odi_csv_male/1080631.csv
ICC Global Cricket Academyin keys
ICC Global Cricket Academy
CricSheet/odi_csv_male/1083446.csv
Rangiri Dambulla International Stadiumnot in keys
Rangiri Dambulla International Stadium
CricSheet/odi_csv_male/1083447.csv
Rangiri Dambulla International Stadiumin keys
CricSheet/odi_csv_male/1083448.csv
Sinhalese Sports Club Groundnot in keys
Sinhalese Sports Club Ground
CricSheet/odi_csv_male/1089183.csv
Sheikh Zayed Stadiumin keys
Sheikh Zayed Stadium
CricSheet/odi_csv_male/1089419.csv
Grange Cricket Club Groundin keys
Grange Cricket Club Ground
CricSheet/odi_csv_male/1089420.csv
Grange Cricket Club Groundin keys
Grange Cricket Club Ground
CricSheet/odi_csv_male/1089779.csv
Darren Sammy National Cricket Stadiumnot in keys
Darren Sammy National Cricket Stadium
CricSheet/odi_csv_male/1089780.csv
Darren Sammy Nati

CricSheet/odi_csv_male/247470.csv
Warner Parkin keys
Warner Park
CricSheet/odi_csv_male/247471.csv
Beausejour Stadiumin keys
Beausejour Stadium
CricSheet/odi_csv_male/247472.csv
Queen's Park Ovalin keys
Queen's Park Oval
CricSheet/odi_csv_male/247473.csv
Sabina Parkin keys
Sabina Park
CricSheet/odi_csv_male/247474.csv
Warner Parkin keys
Warner Park
CricSheet/odi_csv_male/247475.csv
Beausejour Stadiumin keys
Beausejour Stadium
CricSheet/odi_csv_male/247476.csv
Queen's Park Ovalin keys
Queen's Park Oval
CricSheet/odi_csv_male/247477.csv
Sabina Parkin keys
Sabina Park
CricSheet/odi_csv_male/247478.csv
Warner Parkin keys
Warner Park
CricSheet/odi_csv_male/247479.csv
Beausejour Stadiumin keys
Beausejour Stadium
CricSheet/odi_csv_male/247480.csv
Queen's Park Ovalin keys
Queen's Park Oval
CricSheet/odi_csv_male/247481.csv
Sir Vivian Richards Stadiumin keys
Sir Vivian Richards Stadium
CricSheet/odi_csv_male/247482.csv
Providence Stadiumin keys
Providence Stadium
CricSheet/odi_csv_male/247483.c

Multan Cricket Stadiumin keys
Multan Cricket Stadium
CricSheet/odi_csv_male/257772.csv
National Stadiumin keys
National Stadium
CricSheet/odi_csv_male/258465.csv
Lord'sin keys
Lord's
CricSheet/odi_csv_male/258466.csv
Edgbastonin keys
Edgbaston
CricSheet/odi_csv_male/258467.csv
Trent Bridgein keys
Trent Bridge
CricSheet/odi_csv_male/258471.csv
The Rose Bowlin keys
The Rose Bowl
CricSheet/odi_csv_male/258472.csv
County Groundin keys
County Ground
CricSheet/odi_csv_male/258473.csv
Edgbastonin keys
Edgbaston
CricSheet/odi_csv_male/258474.csv
Old Traffordin keys
Old Trafford
CricSheet/odi_csv_male/258475.csv
Headingleyin keys
Headingley
CricSheet/odi_csv_male/258476.csv
Kennington Ovalin keys
Kennington Oval
CricSheet/odi_csv_male/258477.csv
Lord'sin keys
Lord's
CricSheet/odi_csv_male/259384.csv
Sedgars Parkin keys
Sedgars Park
CricSheet/odi_csv_male/259389.csv
Willowmoore Parknot in keys
Willowmoore Park
CricSheet/odi_csv_male/259390.csv
Willowmoore Parkin keys
Willowmoore Park
CricSheet/o

CricSheet/odi_csv_male/299009.csv
Multan Cricket Stadiumin keys
Multan Cricket Stadium
CricSheet/odi_csv_male/299010.csv
Gaddafi Stadiumin keys
Gaddafi Stadium
CricSheet/odi_csv_male/300426.csv
Eden Parkin keys
Eden Park
CricSheet/odi_csv_male/300427.csv
McLean Parkin keys
McLean Park
CricSheet/odi_csv_male/300428.csv
Queenstown Events Centrein keys
Queenstown Events Centre
CricSheet/odi_csv_male/300437.csv
Westpac Stadiumin keys
Westpac Stadium
CricSheet/odi_csv_male/300438.csv
Seddon Parkin keys
Seddon Park
CricSheet/odi_csv_male/300439.csv
Eden Parkin keys
Eden Park
CricSheet/odi_csv_male/300440.csv
McLean Parkin keys
McLean Park
CricSheet/odi_csv_male/300441.csv
AMI Stadiumnot in keys
AMI Stadium
CricSheet/odi_csv_male/300486.csv
Queens Sports Clubin keys
Queens Sports Club
CricSheet/odi_csv_male/300487.csv
Harare Sports Clubin keys
Harare Sports Club
CricSheet/odi_csv_male/300488.csv
Harare Sports Clubin keys
Harare Sports Club
CricSheet/odi_csv_male/313992.csv
Gymkhana Club Groun

...

CricSheet/odi_csv_male/540168.csv
Arnos Vale Groundin keys
Arnos Vale Ground
CricSheet/odi_csv_male/540169.csv
Arnos Vale Groundin keys
Arnos Vale Ground
CricSheet/odi_csv_male/540170.csv
Arnos Vale Groundin keys
Arnos Vale Ground
CricSheet/odi_csv_male/540171.csv
Beausejour Stadiumin keys
Beausejour Stadium
CricSheet/odi_csv_male/540172.csv
Beausejour Stadiumin keys
Beausejour Stadium
CricSheet/odi_csv_male/543881.csv
Mombasa Sports Club Groundnot in keys
Mombasa Sports Club Ground
CricSheet/odi_csv_male/543882.csv
Mombasa Sports Club Groundin keys
Mombasa Sports Club Ground
CricSheet/odi_csv_male/543888.csv
Sharjah Cricket Stadiumin keys
Sharjah Cricket Stadium
CricSheet/odi_csv_male/545656.csv
Sharjah Cricket Stadiumin keys
Sharjah Cricket Stadium
CricSheet/odi_csv_male/555892.csv
Clontarf Cricket Club Groundin keys
Clontarf Cricket Club Ground
CricSheet/odi_csv_male/560923.csv
Sabina Parkin keys
Sabina Park
CricSheet/odi_csv_male/560924.csv
Sabina Parkin keys
Sabina Park
CricSheet/

Queen's Park Oval
CricSheet/odi_csv_male/601077.csv
Grange Cricket Club Groundin keys
Grange Cricket Club Ground
CricSheet/odi_csv_male/601612.csv
Clontarf Cricket Club Groundin keys
Clontarf Cricket Club Ground
CricSheet/odi_csv_male/601613.csv
Clontarf Cricket Club Groundin keys
Clontarf Cricket Club Ground
CricSheet/odi_csv_male/602474.csv
Mahinda Rajapaksa International Cricket Stadiumin keys
Mahinda Rajapaksa International Cricket Stadium
CricSheet/odi_csv_male/602475.csv
Mahinda Rajapaksa International Cricket Stadiumin keys
CricSheet/odi_csv_male/602476.csv
Pallekele International Cricket Stadiumin keys
Pallekele International Cricket Stadium
CricSheet/odi_csv_male/623573.csv
Queens Sports Clubin keys
Queens Sports Club
CricSheet/odi_csv_male/623574.csv
Queens Sports Clubin keys
Queens Sports Club
CricSheet/odi_csv_male/623575.csv
Queens Sports Clubin keys
Queens Sports Club
CricSheet/odi_csv_male/631138.csv
VRA Groundin keys
VRA Ground
CricSheet/odi_csv_male/635653.csv
R Premad

Sheikh Zayed Stadium
CricSheet/odi_csv_male/657645.csv
Sheikh Zayed Stadiumin keys
Sheikh Zayed Stadium
CricSheet/odi_csv_male/659549.csv
Harare Sports Clubin keys
Harare Sports Club
CricSheet/odi_csv_male/659551.csv
Harare Sports Clubin keys
Harare Sports Club
CricSheet/odi_csv_male/659553.csv
Harare Sports Clubin keys
Harare Sports Club
CricSheet/odi_csv_male/660825.csv
Maple Leaf North-West Groundin keys
Maple Leaf North-West Ground
CricSheet/odi_csv_male/660827.csv
Maple Leaf North-West Groundin keys
Maple Leaf North-West Ground
CricSheet/odi_csv_male/660837.csv
Civil Service Cricket Clubin keys
Civil Service Cricket Club
CricSheet/odi_csv_male/661685.csv
Eden Parkin keys
Eden Park
CricSheet/odi_csv_male/661689.csv
Queenstown Events Centrein keys
Queenstown Events Centre
CricSheet/odi_csv_male/661691.csv
Saxton Ovalin keys
Saxton Oval
CricSheet/odi_csv_male/661693.csv
Seddon Parkin keys
Seddon Park
CricSheet/odi_csv_male/667641.csv
McLean Parkin keys
McLean Park
CricSheet/odi_csv_m

Old Trafford
CricSheet/odi_csv_male/743983.csv
Headingleyin keys
Headingley
CricSheet/odi_csv_male/743985.csv
Old Traffordin keys
Old Trafford
CricSheet/odi_csv_male/744679.csv
Shere Bangla National Stadiumin keys
Shere Bangla National Stadium
CricSheet/odi_csv_male/744681.csv
Shere Bangla National Stadiumin keys
Shere Bangla National Stadium
CricSheet/odi_csv_male/744683.csv
Shere Bangla National Stadiumin keys
CricSheet/odi_csv_male/745159.csv
Mahinda Rajapaksa International Cricket Stadiumin keys
Mahinda Rajapaksa International Cricket Stadium
CricSheet/odi_csv_male/745161.csv
Mahinda Rajapaksa International Cricket Stadiumin keys
Mahinda Rajapaksa International Cricket Stadium
CricSheet/odi_csv_male/745163.csv
Rangiri Dambulla International Stadiumin keys
Rangiri Dambulla International Stadium
CricSheet/odi_csv_male/749771.csv
Bay Ovalin keys
Bay Oval
CricSheet/odi_csv_male/749773.csv
Bay Ovalin keys
Bay Oval
CricSheet/odi_csv_male/749775.csv
Seddon Parkin keys
CricSheet/odi_csv_ma

Melbourne Cricket Groundin keys
Melbourne Cricket Ground
CricSheet/odi_csv_male/895813.csv
Manuka Ovalin keys
Manuka Oval
CricSheet/odi_csv_male/895815.csv
Sydney Cricket Groundin keys
Sydney Cricket Ground
CricSheet/odi_csv_male/902641.csv
Sheikh Zayed Stadiumin keys
Sheikh Zayed Stadium
CricSheet/odi_csv_male/902643.csv
Sheikh Zayed Stadiumin keys
Sheikh Zayed Stadium
CricSheet/odi_csv_male/902645.csv
Sharjah Cricket Stadiumin keys
Sharjah Cricket Stadium
CricSheet/odi_csv_male/902647.csv
Dubai International Cricket Stadiumin keys
Dubai International Cricket Stadium
CricSheet/odi_csv_male/903593.csv
Green Parkin keys
Green Park
CricSheet/odi_csv_male/903595.csv
Holkar Cricket Stadiumin keys
Holkar Cricket Stadium
CricSheet/odi_csv_male/903597.csv
Saurashtra Cricket Association Stadiumin keys
Saurashtra Cricket Association Stadium
CricSheet/odi_csv_male/903599.csv
MA Chidambaram Stadiumin keys
MA Chidambaram Stadium
CricSheet/odi_csv_male/903601.csv
Wankhede Stadiumin keys
Wankhede St

In [11]:
avgVenueMap = dict()
avgList = []

for venue in venueMap.keys():
    avg = sum(venueMap[venue])/len(venueMap[venue])
    avgVenueMap[venue] = avg
#     print(avg)
#     print(venueMap[venue])
    

for index, row in df.iterrows(): 
#     print(index)
#     print(row["Venue"])
    venue = row["Venue"]
    avgList.append(avgVenueMap[venue])

df["VenueAvg"] = avgList
    


cols = [
 'Game ID',
 'Innings',
 'Bat Team',
 'Batsman',
 'Non_Striker',
 'Bowler',
 'BatsmanScore',
 'Extras',
 'Total Runs*',
 'Over*',
 'Ball*',
 'Runs*',
 'Run rate*',
 'Wickets*',
 'Total balls*',
    'Venue',
    'VenueAvg'
]
df = df[cols]





In [12]:
df.to_csv('Sample.csv')

In [35]:
feature_cols = ['BatsmanScore','Over*', 'Run rate*', 'Wickets*', 'VenueAvg', 'Total balls*', 'Innings']
target_cols =['Total Runs*']

features_X = df[feature_cols]
Y = df[target_cols]



In [17]:
model = LogisticRegression(penalty = "l2")
kfold = KFold(n_splits=3, random_state=42)
results = cross_val_score(model, features_X, Y, cv=kfold)

In [18]:
print(results)

[0.02752464 0.02845453 0.02417705]


In [37]:
param_test1 = {
    'max_depth':range(3,10,1),
    'min_child_weight':range(0,3,1)
}
gsearch1 = GridSearchCV(estimator = XGBRegressor( 
    learning_rate =0.1, 
    n_estimators=300, 
    max_depth=32,
    min_child_weight=0,
    gamma=0.0, 
    subsample=0.8, 
    colsample_bytree=0.15,
#     objective= 'binary:logistic', 
    nthread=16, 
    scale_pos_weight=1, 
    seed=42,
    reg_lambda=0
), 
param_grid = param_test1,n_jobs=-1,iid=False, cv=10)
gsearch1.fit(features_X,Y)
gsearch1.cv_results_, gsearch1.best_params_, gsearch1.best_score_

({'mean_fit_time': array([2.94386168, 2.80023007, 2.90325427, 3.47018018, 3.43117228,
         3.44537511, 3.76224594, 3.7116364 , 3.82065988, 4.12932849,
         4.02150512, 4.03570843, 4.33957667, 4.25155668, 4.32437339,
         4.58463149, 4.31917086, 4.32117295, 4.41239281, 3.53159389,
         3.28633919]),
  'std_fit_time': array([0.07436292, 0.02347145, 0.04996609, 0.0696197 , 0.09163712,
         0.04551804, 0.13650986, 0.01604586, 0.03705145, 0.14255966,
         0.02269126, 0.06492486, 0.08082496, 0.02610966, 0.04314044,
         0.04061706, 0.0317593 , 0.04983236, 0.12350207, 0.25347342,
         0.06746753]),
  'mean_score_time': array([0.01280336, 0.01340303, 0.01820388, 0.01200333, 0.0094018 ,
         0.01080332, 0.01180377, 0.01140237, 0.01480346, 0.01360378,
         0.01180334, 0.01320333, 0.0120028 , 0.01320319, 0.01280289,
         0.012603  , 0.01240339, 0.01600366, 0.01220269, 0.01320314,
         0.01860414]),
  'std_score_time': array([0.00487544, 0.00531489, 

In [38]:
param_test2 = {
    #gamma':[i/100 for i in range(0,6,1)]
#     'gamma':[0.01,0.02,0,0.03,0.001,0.0001,0.00001]
    'gamma':[.0001,.00001,0]
}
gsearch1 = GridSearchCV(estimator = XGBRegressor( 
    learning_rate =0.1, 
    n_estimators=300, 
    max_depth=4,
    min_child_weight=0,
    gamma=0.0, 
    subsample=0.8, 
    colsample_bytree=0.15,
#     objective= 'binary:logistic', 
    nthread=16, 
    scale_pos_weight=1, 
    seed=42,
    reg_alpha=0,
    reg_lambda = 0

), 
param_grid = param_test2,n_jobs=-1,iid=False, cv=10)
gsearch1.fit(features_X,Y)
gsearch1.cv_results_, gsearch1.best_params_, gsearch1.best_score_

({'mean_fit_time': array([3.33715086, 3.24113011, 3.22212529]),
  'std_fit_time': array([0.11024995, 0.04492141, 0.04287991]),
  'mean_score_time': array([0.01820402, 0.01560354, 0.01620374]),
  'std_score_time': array([0.00879703, 0.00523939, 0.00691241]),
  'param_gamma': masked_array(data=[0.0001, 1e-05, 0],
               mask=[False, False, False],
         fill_value='?',
              dtype=object),
  'params': [{'gamma': 0.0001}, {'gamma': 1e-05}, {'gamma': 0}],
  'split0_test_score': array([0.96981181, 0.96981181, 0.96981181]),
  'split1_test_score': array([0.96956319, 0.96956319, 0.96956319]),
  'split2_test_score': array([0.96462221, 0.96462221, 0.96462221]),
  'split3_test_score': array([0.9663063, 0.9663063, 0.9663063]),
  'split4_test_score': array([0.95015032, 0.95015032, 0.95015032]),
  'mean_test_score': array([0.96409077, 0.96409077, 0.96409077]),
  'std_test_score': array([0.00724161, 0.00724161, 0.00724161]),
  'rank_test_score': array([1, 1, 1]),
  'split0_train_sc

In [44]:
param_test3 = {
    'subsample':[i/100 for i in range(75,82,1)],
    'colsample_bytree':[i/100.0 for i in range(0,6,1)]
}
gsearch1 = GridSearchCV(estimator = XGBRegressor(
    learning_rate =0.1, 
    n_estimators=300, 
    max_depth=4,
    min_child_weight=0, 
    gamma=0.0001, 
    subsample=0.82, 
    colsample_bytree=0.22,
#     objective= 'binary:logistic', 
    nthread=16, 
    scale_pos_weight=1, 
    seed=42,
    reg_alpha = 0, 
    reg_lambda = 1
), 
param_grid = param_test3,n_jobs=-1,iid=False, cv=10)
gsearch1.fit(features_X,Y)
gsearch1.cv_results_, gsearch1.best_params_, gsearch1.best_score_

({'mean_fit_time': array([3.64031804, 3.68823106, 3.72163794, 3.68403006, 3.69593267,
         3.74414322, 3.77835095, 3.79145393, 3.66032441, 3.69433186,
         3.6877295 , 3.69453225, 3.73254077, 3.62691698, 3.65532351,
         3.68393009, 3.6460212 , 3.66742599, 3.65032198, 3.64152069,
         3.65182214, 3.63881936, 3.65932431, 3.67562778, 3.64472098,
         3.6960324 , 3.78795307, 3.70003324, 3.70293376, 3.66122463,
         3.72934039, 3.74534333, 3.63771915, 3.73254063, 3.8705714 ,
         3.9762954 , 3.96299257, 4.16253765, 3.83546371, 3.86657073,
         3.78585234, 2.81963468]),
  'std_fit_time': array([0.11552461, 0.09435382, 0.07404291, 0.08414945, 0.03986474,
         0.02768311, 0.07446045, 0.03801436, 0.04003939, 0.02596805,
         0.03967428, 0.03315278, 0.08018933, 0.02541637, 0.03252575,
         0.0443689 , 0.02325921, 0.06587245, 0.03753518, 0.04144166,
         0.03170556, 0.01327722, 0.06694516, 0.03835375, 0.02377241,
         0.02656864, 0.03014452, 0.

In [46]:
param_test4 = {
    'reg_alpha':[0,0.1,1,0.01]
    #'reg_alpha':[0.1,0.2,0,1,2]
}
gsearch1 = GridSearchCV(estimator = XGBRegressor( 
    learning_rate =0.1, 
    n_estimators=300, 
    max_depth=20,
    min_child_weight=4,
    gamma=0.0, 
    subsample=0.81, 
    colsample_bytree=0.0,
#     objective= 'binary:logistic', 
    nthread=16, 
    scale_pos_weight=1, 
    seed=42,
    reg_lambda=0
    
), 
param_grid = param_test4,n_jobs=-1,iid=False, cv=10)
gsearch1.fit(features_X,Y)
gsearch1.cv_results_, gsearch1.best_params_, gsearch1.best_score_

({'mean_fit_time': array([7.22630553, 6.71701984, 5.67367795, 4.06011369]),
  'std_fit_time': array([0.31191325, 0.7334568 , 0.23326128, 0.87930226]),
  'mean_score_time': array([0.01300311, 0.01420341, 0.01130276, 0.01000247]),
  'std_score_time': array([0.00562245, 0.0077579 , 0.00659023, 0.0017894 ]),
  'param_reg_alpha': masked_array(data=[0, 0.1, 1, 0.01],
               mask=[False, False, False, False],
         fill_value='?',
              dtype=object),
  'params': [{'reg_alpha': 0},
   {'reg_alpha': 0.1},
   {'reg_alpha': 1},
   {'reg_alpha': 0.01}],
  'split0_test_score': array([0.97005906, 0.97005056, 0.97002646, 0.97006017]),
  'split1_test_score': array([0.95957395, 0.95956624, 0.95949204, 0.95957251]),
  'split2_test_score': array([0.96576337, 0.96578469, 0.96575627, 0.96576309]),
  'split3_test_score': array([0.89277653, 0.89273431, 0.89245924, 0.89277176]),
  'split4_test_score': array([0.9336902 , 0.93368586, 0.93371723, 0.93368757]),
  'split5_test_score': array([0.

In [50]:
model = XGBRegressor(
    learning_rate =0.03, 
    n_estimators=5000, 
    max_depth=4,
    min_child_weight=0, 
    gamma=0.0001, 
    subsample=0.81, 
    colsample_bytree=0.0,
#     objective= 'reg:squarederror', 
    nthread=16, 
    scale_pos_weight=1, 
    seed=42,
    reg_alpha = 0, 
    reg_lambda = 1
)

model.fit(features_)
plot_importance(model)
pyplot.show()
kfold = KFold(n_splits=10, random_state=42)
results = cross_val_score(model, features_X, Y, cv=kfold)
print(results)
print(results.mean())
print("Accuracy: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

XGBoostError: need to call fit or load_model beforehand